In [1]:
import os    

os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
import numpy as np
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt
%matplotlib inline


import theano
from keras.layers import Input, Dense
from keras.layers import Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer, Merge
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.utils.np_utils import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score


Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
Using Theano backend.
/home/mrafi123/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Get the raw data
x_train = np.genfromtxt('x_train.out')
y_train = np.genfromtxt('y_train.out')
vx_train = np.genfromtxt('vx_train.out')
vy_train = np.genfromtxt('vy_train.out')
x_test = np.genfromtxt('x_test.out')
y_test = np.genfromtxt('y_test.out')

In [3]:
print (x_train.shape)
print (y_train.shape)
print (vx_train.shape)
print (vy_train.shape)
print (x_test.shape)
print (y_test.shape)

(50000, 784)
(50000,)
(10000, 784)
(10000,)
(10000, 784)
(10000,)


In [4]:
label_train=to_categorical(y_train)
label_valid=to_categorical(vy_train)
label_test= to_categorical(y_test)

/home/mrafi123/anaconda3/lib/python3.5/site-packages/keras/utils/np_utils.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y = np.zeros((len(y), nb_classes))
/home/mrafi123/anaconda3/lib/python3.5/site-packages/keras/utils/np_utils.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


In [5]:
print (x_train.shape)
print (label_train.shape)
print (vx_train.shape)
print (label_valid.shape)
print (x_test.shape)
print (label_test.shape)

(50000, 784)
(50000, 10)
(10000, 784)
(10000, 10)
(10000, 784)
(10000, 10)


In [6]:
# reshape data for CNN 

#train_x_temp = x_train.reshape(-1, 28, 28, 1)
#val_x_temp = vx_train.reshape(-1, 28, 28, 1)
#test_x_temp=x_test.reshape(-1, 28, 28, 1)

train_x_temp = x_train.reshape(-1,1, 28, 28)
val_x_temp = vx_train.reshape(-1,1, 28, 28)
test_x_temp=x_test.reshape(-1,1, 28, 28)
print(train_x_temp.shape)
print(val_x_temp.shape)
print(test_x_temp.shape)

(50000, 1, 28, 28)
(10000, 1, 28, 28)
(10000, 1, 28, 28)


In [67]:
# define model parameters and model architecture
input_shape = (784,)
input_reshape1 = (1,28, 28)
conv_num_filters = 25
conv_filter_size = 5
pool_size = (2, 2)
hidden_num_units = 50
output_num_units = 10






In [93]:
# this is our input placeholder
# refer architecture from DCCNN paper
left_branch1 = Sequential()
left_branch1.add(Convolution2D(64, 5, 5, border_mode='same',activation='relu',input_shape=input_reshape1))
right_branch1 = Sequential()
right_branch1.add(Convolution2D(64, 5, 5, border_mode='same',activation='relu',input_shape=input_reshape1))
merged1 = Merge([left_branch1, right_branch1], mode='ave')


model1 = Sequential()
model1.add(merged1)
model1.add(MaxPooling2D(pool_size=pool_size))


In [99]:
print ('Model 1',model1.summary())


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_143 (Convolution2D)(None, 64, 28, 28)    1664        convolution2d_input_141[0][0]    
____________________________________________________________________________________________________
convolution2d_144 (Convolution2D)(None, 64, 28, 28)    1664        convolution2d_input_142[0][0]    
____________________________________________________________________________________________________
maxpooling2d_66 (MaxPooling2D)   (None, 64, 14, 14)    0           merge_78[0][0]                   
Total params: 3328
____________________________________________________________________________________________________
Model 1 None


In [95]:
input_reshape2 = (64, 14, 14)
left_branch2 = Sequential()
left_branch2.add(Convolution2D(128, 4, 4, border_mode='same',activation='relu',input_shape=input_reshape2))
#left_branch2.add(Convolution2D(128, 4, 4, border_mode='same',activation='relu')) - error

right_branch2 = Sequential()
right_branch2.add(Convolution2D(128, 4, 4, border_mode='same',activation='relu',input_shape=input_reshape2))
#right_branch2.add(Convolution2D(128, 4, 4, border_mode='same',activation='relu')) - error

merged2 = Merge([left_branch2, right_branch2], mode='ave')
model2 = Sequential()
model2.add(merged2)
model2.add(MaxPooling2D(pool_size=pool_size))



In [98]:
print ('Model 2',model2.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_145 (Convolution2D)(None, 128, 14, 14)   131200      convolution2d_input_143[0][0]    
____________________________________________________________________________________________________
convolution2d_146 (Convolution2D)(None, 128, 14, 14)   131200      convolution2d_input_144[0][0]    
____________________________________________________________________________________________________
maxpooling2d_67 (MaxPooling2D)   (None, 128, 7, 7)     0           merge_79[0][0]                   
Total params: 262400
____________________________________________________________________________________________________
Model 2 None


In [100]:
model = Sequential()
model.add(model1)
model.add(Flatten())
model.add(model2)
model.add(Flatten())
model.add(Dense(output_dim=output_num_units, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Exception: Layer sequential_249 expects 2 inputs, but it received 1 input tensors. Input received: Reshape{2}.0

In [ ]:
nfold=5
nb_epoch=50
batch_size=256
random_state =335

In [ ]:
kfold_weights_path = os.path.join('mnist_weights_kfold_' + str(nfold) + 
                                  '_epoch_'+str(nb_epoch)+
                                  '_batch_'+str(batch_size) +
                                  '_encoded_'+str(encoding_dim) +
                                  '.h5')
print(kfold_weights_path)

In [ ]:
kf = KFold(len(x_train), n_folds=nfold, shuffle=True, random_state=random_state)
#kf = KFold(len(x_train), n_folds=nfold, shuffle=False, random_state=random_state)
#Shuffle false give very high training loss and very low val loss - indicating unknown fit

print(kf)

In [ ]:
train_full_encoded_imgs = np.zeros(shape=[x_train.shape[0],encoding_dim])
print(train_full_encoded_imgs.shape)
print(x_test.shape)
test_full_encoded_imgs = np.zeros(shape=[x_test.shape[0],encoding_dim])
test_full_decoded_imgs = np.zeros(shape=[x_test.shape[0],x_test.shape[1]])
print(test_full_encoded_imgs.shape)
print(test_full_decoded_imgs.shape)

In [ ]:
os.path.isfile(kfold_weights_path)

In [ ]:
# Enable this for avoiding using pretrained weights
if os.path.isfile(kfold_weights_path):
    print ('Loading weights already stored...')
    model.load_weights(kfold_weights_path)

In [ ]:
num_fold = 0
restore_from_last_checkpoint=0
for train_index,valid_index in kf:
    s_train,s_valid  = x_train[train_index], x_train[valid_index]
    
    noise_factor = 0.3
    s_train_noisy = s_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=s_train.shape) 
    s_valid_noisy = s_valid + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=s_valid.shape) 

    s_train_noisy = np.clip(s_train_noisy, 0., 1.)
    s_valid_noisy = np.clip(s_valid_noisy, 0., 1.)
    #y_train, y_test = y[train_index], y[test_index]
    
    num_fold += 1
    print('\n\nStart KFold number {} from {}'.format(num_fold, nfold))
    print('Split train: ', len(s_train_noisy), len(s_train))
    print('Split valid: ', len(s_valid_noisy), len(s_valid))

    if not os.path.isfile(kfold_weights_path) or restore_from_last_checkpoint == 0:
        callbacks = [
                EarlyStopping(monitor='val_loss', patience=3, verbose=0),
                ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0),
        ]
        trained_model_conv = model.fit(train_x_temp, label_train, 
                               nb_epoch=epochs, 
                               batch_size=batch_size,
                               shuffle=True,
                               validation_data=(val_x_temp, label_valid),
                               callbacks=callbacks)
        
      
    # Store train predictions
    train_encoded_imgs = encoder.predict(x_train,batch_size=batch_size, verbose=1)
    train_full_encoded_imgs = np.add(train_full_encoded_imgs,train_encoded_imgs)

    
    # Store test predictions
    test_encoded_imgs = encoder.predict(x_test,batch_size=batch_size, verbose=1)
    #full_encoded_imgs = np.vstack([full_encoded_imgs,encoded_imgs])
    test_full_encoded_imgs = np.add(test_full_encoded_imgs,test_encoded_imgs)
    #full_encoded_imgs.append(encoded_imgs)
    #print(full_encoded_imgs.shape)
    
    test_decoded_imgs = decoder.predict(test_encoded_imgs,batch_size=batch_size, verbose=1)
    #full_decoded_imgs = np.vstack([full_decoded_imgs,decoded_imgs])
    test_full_decoded_imgs = np.add(test_full_decoded_imgs,test_decoded_imgs)

In [ ]:
test_res_encoded_imgs = test_full_encoded_imgs/nfold
test_res_decoded_imgs = test_full_decoded_imgs/nfold
print(test_res_encoded_imgs.shape)
print(test_res_decoded_imgs.shape)
print(test_res_encoded_imgs)
print(test_res_decoded_imgs)
train_res_encoded_imgs = train_full_encoded_imgs/nfold
print(train_res_encoded_imgs.shape)
print(train_res_encoded_imgs)

In [ ]:
#list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    #ax = plt.subplot(1, n, i)
    plt.imshow(s_train_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [41]:
#pred = model.predict_classes(x_test)
pred = model.predict_classes(test_x_temp)

 9984/10000 [============================>.] - ETA: 0s

In [42]:
pred

array([7, 2, 1, ..., 4, 5, 6])

In [43]:
pred= to_categorical(pred)

In [44]:
pred

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [45]:
label_test

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [46]:
num=len(label_test)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(pred[i],label_test[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")

tested  10000 digits
correct:  9877 wrong:  123 error rate:  1.23 %
got correctly  98.77 %
